# OIC-158 TEM Nuclei Segmentation
Packages to import

In [1]:
import napari
import numpy as np
import pyclesperanto as cle
import skimage as sk
from skimage.io import imread, imsave
import pandas as pd
import os
from glob import glob
import matplotlib as mpl
import matplotlib.pyplot as plt

Load in data and filter out masks touching border and small labeled areas created during ground truth annotation

In [ ]:
img_files = sorted(glob("E:/Fondufe-Mittendorf_Lab/TEM_Images/Images/*.tif"))
mask_files = sorted(glob("E:/Fondufe-Mittendorf_Lab/TEM_Images/Masks/*.tif"))
imgs = list(map(imread,img_files))
imgs = [np.asarray(img,dtype=np.uint16) for img in imgs] #make sure all imgs are np compatible arrays
masks = list(map(imread,mask_files))
masks = [np.asarray(mask,dtype=np.uint16) for mask in masks] #make sure all label images are compatible arrays
filtered_masks = [sk.segmentation.clear_border(sk.morphology.remove_small_objects(mask, min_size=50)) for mask in masks]
relabel_masks = [sk.measure.label(mask) for mask in filtered_masks]

Create 25-pixel diameter band around nuclear membrane

In [ ]:
def band_25_pixels(footprint = 25, masks):
    eroded = [sk.morphology.erosion(mask, footprint=footprint) for mask in masks]
    bands = [masks[i] - eroded[i] for i in range(len(masks))]
    return bands

In [ ]:
bands_25 = band_25_pixels(masks=relabel_masks)

### testing cells for creating band

In [ ]:
#erode objects to get a thin outer border (disc size 25 is better option)
footprint_50 = sk.morphology.disk(50)
eroded = sk.morphology.erosion(test_mask,footprint_50)
band_50 = test_mask - eroded
# viewer = napari.view_image(relabeled)
# viewer.add_labels(eroded)
# viewer.add_labels(band_50)

In [ ]:
test_mask = filtered_masks[11]
footprint_25 = sk.morphology.disk(25)
eroded = sk.morphology.erosion(test_mask,footprint_25)
band_25 = test_mask - eroded
# viewer = napari.view_image(imgs[11])
# viewer.add_labels(eroded)
# viewer.add_labels(band_25, name='25')

## Thresholding with Yen

In [44]:
#footprint = sk.morphology.disk(2)
adpt_norm = [sk.exposure.equalize_adapthist(img) for img in imgs]
matched_hist = [sk.exposure.match_histograms(img, adpt_norm[3]) for img in adpt_norm]
filtered = []
for img in matched_hist:
    yen = sk.filters.threshold_yen(img)
    mask = img < yen
    dilate = sk.morphology.dilation(mask)
    filtered.append(sk.morphology.remove_small_objects(sk.morphology.remove_small_holes(dilate,area_threshold=100),min_size=20))

c:\ProgramData\anaconda3\envs\CellPoseCLE-env\lib\site-packages\skimage\filters\thresholding.py:466: RuntimeWarning: divide by zero encountered in log
  crit = np.log(((P1_sq[:-1] * P2_sq[1:]) ** -1) * (P1[:-1] * (1.0 - P1[:-1])) ** 2)


In [43]:
stack = np.stack(filtered)
viewer = napari.view_labels(stack)

## Get area of band covered by dense signal

In [ ]:
mask_dense_nuclei = [filtered[i] * relabel_masks[i] for i in range(len(filtered))]
#mask_dense_nuclei_stack = np.stack(mask_dense_nuclei)
dense_band = [mask_dense_nuclei[i] * bands[i] for i in range(len(mask_dense_nuclei))]
#dense_band_stack = np.stack(dense_band)



Merge the image and all masks into arrays

In [50]:
merged_stacks = []
for i in range(len(imgs)):
    stack = np.stack([imgs[i],relabel_masks[i],bands[i],mask_dense_nuclei[i],dense_band[i]],axis=-1)
    merged_stacks.append(stack)

In [51]:
merged_stacks[0].shape

(2672, 4008, 5)

## Get measurements

In [48]:
props = ['label', 'area', 'eccentricity', 'equivalent_diameter_area','extent','perimeter','solidity','intensity_max',]
scale = [0.0064,0.0064]

In [133]:
def heterochromatin_area(heterochromatin_band, bands, scale):
    areas = []
    for i in range(1,np.max(bands)):
        band = bands == i
        mask = band * heterochromatin_band
        area = (np.count_nonzero(mask) * scale[0] * scale[1])
        areas.append(area)
    areas_df = pd.Series(areas, name='heterochromatin_area_um^2')
    return areas, areas_df


In [134]:
areas, areas_df = heterochromatin_area(heterochromatin_band=dense_band[24],bands=bands[24],scale=scale)
areas

[1.2183142400000002, 0.7679180800000001, 3.0156390400000004, 1.14778112]

In [138]:
def get_measurements(img_stack,props,scale,heterochromatin_areas, heterochromatin_area_df):
    nuc_measurements_table = sk.measure.regionprops_table(img_stack[:,:,1],img_stack,props,spacing=scale)
    nuc_regionprops = sk.measure.regionprops(img_stack[:,:,1],spacing=scale)
    band_25_measurements= sk.measure.regionprops_table(img_stack[:,:,2],properties=('label','area'),spacing=scale)
    band_25_df = pd.DataFrame.from_dict(band_25_measurements)
    band_25_df = band_25_df.rename(columns={'label':'Band_ID','area':'area_um^2'})
    circularity = []
    for i in range(len(nuc_regionprops)):
        nuc_area = nuc_regionprops[i].area
        nuc_perimeter = nuc_regionprops[i].perimeter
        circularity.append(4*np.pi*nuc_area/nuc_perimeter**2)
    circularity = pd.Series(circularity,name='nuc_circularity')
    percent_band_covered = []
    band_25_regionprops= sk.measure.regionprops(img_stack[:,:,2],spacing=scale)
    for i in range(len(heterochromatin_areas)):
        band_area = band_25_regionprops[i].area
        percent_covered = heterochromatin_areas[i]/band_area
        percent_band_covered.append(percent_covered)
    percent_band_covered = pd.Series(percent_band_covered, name='norm_area_heterochromatin_band')
    nuc_df = pd.DataFrame.from_dict(nuc_measurements_table)
    nuc_df.rename(columns={'label':'cell_ID','area':'Nuc_area_um^2','intensity_max-0':'Image_intensity_max',
                           'intensity_max-1':'Nuc_ID','intensity_max-2':'Band_ID','intensity_max-3':'Dense_nuc_regions_ID',
                           'intensity_max-4':'Heterochromatin_Band_ID','equivalent_diameter_area':'equivalent_diameter_area_um^2',
                           'perimeter':'perimeter_um'},inplace=True)
    df = pd.concat([nuc_df,circularity,heterochromatin_area_df,percent_band_covered,band_25_df], axis=1)
    return df


In [139]:
test_measurements = get_measurements(merged_stacks[24],props,scale,heterochromatin_areas=areas,heterochromatin_area_df=areas_df)

In [140]:
test_measurements.head()

,cell_ID,Nuc_area_um^2,eccentricity,equivalent_diameter_area_um^2,extent,perimeter_um,solidity,Image_intensity_max,Nuc_ID,Band_ID,Dense_nuc_regions_ID,Heterochromatin_Band_ID,nuc_circularity,heterochromatin_area_um^2,norm_area_heterochromatin_band,Band_ID,area_um^2
0,1,9.531720,0.728778,3.483700,0.542944,16.719343,0.801982,16603.0,1.0,1.0,1.0,1.0,0.428492,1.218314,0.496254,1,2.455020
1,2,8.933868,0.563816,3.372677,0.642898,12.540571,0.923839,15908.0,2.0,2.0,2.0,4.0,0.713863,0.767918,0.423799,2,1.811988
2,3,48.830546,0.612653,7.884985,0.696099,38.268035,0.958699,17152.0,3.0,3.0,3.0,9.0,0.419015,3.015639,0.528794,3,5.702861
3,4,9.923912,0.733061,3.554647,0.541110,17.780411,0.811772,16100.0,4.0,4.0,4.0,16.0,0.394466,1.147781,0.437721,4,2.622177
4,5,1.050870,0.894760,1.156723,0.560370,4.492404,0.948957,15973.0,5.0,5.0,5.0,25.0,0.654336,NaN,NaN,5,0.604733


In [ ]:
viewer = napari.view_image(images_stack, name='img')
viewer.add_labels(label_stack, name='nuclei mask')
viewer.add_labels(bands_stack,name='25px band')
viewer.add_labels(mask_dense_nuclei_stack,name='dense areas in nuc')
viewer.add_labels(dense_band_stack, name='heterochromatin/')


<Labels layer 'heterochromatin' at 0x1d0b08e2860>

### Multi-Otsu testing
4 classes with preprocessing of gaussian sigma=2; use with ground truth masks

Use to measure the dense heterochromatin along the membrane and the nucleoli within the nuclei

In [ ]:
#find threshold or approach that works well for segmenting the dense areas
#multi-otsu
mask = test_mask == 1
gauss = sk.filters.gaussian(imgs[11], sigma=2)
masked_img = mask * gauss
o1,o2,o3 = sk.filters.threshold_multiotsu(image=masked_img, classes=4)



In [ ]:
otsu1 = (masked_img < o1)*mask
otsu2 = (masked_img < o2)*mask
otsu3 = (masked_img < o3)*mask

In [ ]:
viewer = napari.view_image(imgs[11], name='image')
viewer.add_labels(otsu1, name='otsu1')
viewer.add_labels(otsu2, name='otsu2')
viewer.add_labels(otsu3, name='otsu3')

In [ ]:
test_img = imgs[24]
test_img = np.asarray(test_img,dtype=np.uint16)

## Test histogram norm with histogram matching

In [ ]:
# Correct uneven illumination

adpt_norm = sk.exposure.equalize_adapthist(test_img)
norm = sk.exposure.equalize_hist(test_img)
# viewer = napari.view_image(test_img)
# viewer.add_image(adpt_norm, name='adapt norm')
# viewer.add_image(norm, name='norm')

In [ ]:
# add in gamma and contrast adjustment to enhance dense regions; didn't make a large difference for thresholding
gamma_adjust = sk.exposure.adjust_gamma(adpt_norm, gamma=1.5)
viewer = napari.view_image(adpt_norm)
viewer.add_image(gamma_adjust, name='gamma')

In [ ]:
# some images fail on ostu multiclass, testing if matching histogram to images that have successful ostu works


## Testing Sobel and Canny for segmenting the nuclei

### Canny Testing
Close! But not enough connectivity with the lines from canny


In [ ]:
footprint = sk.morphology.disk(6)
top_hat = sk.morphology.black_tophat(adpt_norm)
viewer = napari.view_image(top_hat)

In [ ]:
edges = sk.feature.canny(gamma_adjust, sigma=2.4)
viewer = napari.view_image(adpt_norm)
viewer.add_image(edges, name='edges')

## Sobel Testing
Not the best approach for TEM images. Not enough contrast in the staining for sobel to find the features of interest

In [ ]:
edges = sk.filters.sobel(adpt_norm)
viewer = napari.view_image(adpt_norm)
viewer.add_image(edges, name='edges')

## Multi-Otsu on histogram equalized images
Use second otsu value

In [ ]:
#test image and masks
test_mask = filtered_masks[24]
test_img = np.asarray(imgs[24],dtype=np.uint16)
adpt_norm = sk.exposure.equalize_adapthist(test_img)

In [ ]:
fig, ax = sk.filters.try_all_threshold(adpt_norm)

In [ ]:
#trying yen threshold
mask = test_mask == 1
gauss = sk.filters.gaussian(adpt_norm, sigma=1)
masked_img = mask * gauss
yen = sk.filters.threshold_yen(image=gauss)
yen_mask = adpt_norm < yen
viewer = napari.view_image(adpt_norm)
viewer.add_labels(yen_mask)

In [ ]:
#trying triangle threshold
mask = test_mask == 1
gauss = sk.filters.gaussian(adpt_norm, sigma=1)
masked_img = mask * gauss
triangle = sk.filters.threshold_triangle(image=gauss)
triangle_mask = adpt_norm < triangle
viewer = napari.view_image(adpt_norm)
viewer.add_labels(triangle_mask)

In [ ]:
mask = test_mask == 1
gauss = sk.filters.gaussian(adpt_norm, sigma=1)
masked_img = mask * gauss
o1,o2,o3 = sk.filters.threshold_multiotsu(image=masked_img, classes=4)
# otsu1 = (masked_img < o1)*mask
otsu2 = (masked_img < o2)*mask
# otsu3 = (masked_img < o3)*mask
viewer = napari.view_image(adpt_norm, name='image')
#viewer.add_labels(otsu1, name='otsu1')
viewer.add_labels(otsu2, name='otsu2')
#viewer.add_labels(otsu3, name='otsu3')

In [ ]:
labeled_img = sk.morphology.label(otsu2)
props = ['label','area']
region_props = sk.measure.regionprops_table(labeled_img,properties=props)
df = pd.DataFrame.from_dict(region_props)
areas = np.asarray(df['area'])

In [ ]:
fig, ax = plt.subplots()
ax.hist(areas,bins=100,range=[0,2000])
ax.set_yscale('log')
plt.show()

In [ ]:
#filter out small objects and fill holes
filtered = sk.morphology.remove_small_objects(sk.morphology.remove_small_holes(otsu2,area_threshold=10000),min_size=500)
viewer = napari.view_image(adpt_norm, name='img')
viewer.add_labels(filtered, name='labels')
viewer.add_labels(otsu2, name='otsu')

In [ ]:
viewer.add_labels(band_25,name='band')

## Adding in measurements of heterochromatin in band and nucleoli in nuclei

In [ ]:
view